In [8]:
import tensorflow_datasets as tfds
import pandas as pd

# Data Source

`movielens/1m-ratings`: This dataset contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens. It is the largest dataset that contains demographic data.
Each user has rated at least 20 movies. Ratings are in whole-star increments. In demographic data, age values are divided into ranges and the lowest age value for each range is used in the data instead of the actual values.

`movielens/1m-movies`: This dataset contains data of approximately 3,900 movies rated in the 1m dataset.

# Load data

In [15]:
'''
load the MovieLens 1M dataset, it takes around 2 minutes to download
1m dataset is the largest MovieLens dataset that contains demographic data
if not downloaded, it will download the dataset; otherwise, it will load the dataset
'''
# data = tfds.load("movielens/1m-ratings")
# df = tfds.as_dataframe(data['train'])
# df.to_csv("data/raw/movielens_1m_ratings.csv", index=False)

# load the dataset from the csv file
df = pd.read_csv("data/raw/movielens_1m_ratings.csv")
print(df.shape)
df.head(5)

(1000209, 11)


,bucketized_user_age,movie_genres,movie_id,movie_title,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,35.0,[0 7],b'3107',b'Backdraft (1991)',977432193,True,b'130',18,b'technician/engineer',5.0,b'50021'
1,25.0,[7],b'2114',"b'Outsiders, The (1983)'",965932967,False,b'3829',0,b'academic/educator',4.0,b'22307'
2,18.0,[ 4 15],b'256',b'Junior (1994)',1012103552,False,b'1265',21,b'writer',1.0,b'49321'
3,18.0,[ 0 10],b'1389',b'Jaws 3-D (1983)',972004605,True,b'2896',14,b'sales/marketing',5.0,b'60073'
4,18.0,[0],b'3635',"b'Spy Who Loved Me, The (1977)'",961180111,True,b'5264',17,b'college/grad student',4.0,b'15217'


(1000209, 11)


In [18]:
'''
Load the MovieLens 1M movies dataset.
It contains data of approximately 3,900 movies rated in the 1m dataset.
If not downloaded, it will download the dataset; otherwise, it will load the dataset.
'''
# movie_data = tfds.load("movielens/1m-movies")
# movie_df = tfds.as_dataframe(movie_data['train'])
# movie_df.to_csv("data/raw/movielens_1m_movies.csv", index=False)

# load the dataset from the csv file
movie_df = pd.read_csv("data/raw/movielens_1m_movies.csv")
print(movie_df.shape)
movie_df.head(5)

(3883, 3)


,movie_genres,movie_id,movie_title
0,[5 7],b'1729',b'Jackie Brown (1997)'
1,[7],b'1486',"b'Quiet Room, The (1996)'"
2,[4],b'3086',b'March of the Wooden Soldiers (a.k.a. Laurel ...
3,[0],b'2965',"b'Omega Code, The (1999)'"
4,[10],b'2853',"b'Communion (a.k.a. Alice, Sweet Alice/Holy Te..."
